In [ ]:
!pip install lime
!pip install lifelines

 **Using LIME on the COMPAS dataset**

In this experiment we used the LIME model explanation tool to understand the results given by the model. the model inquestion is the Non-Linear Support Vector Machine model. the dataset used in this model is the COMPAS dataset provided by propublica's github.

First import the dependencies

In [ ]:
from __future__ import print_function
import pandas as pd
import numpy as np
import lime
import lime.lime_tabular
import sklearn.ensemble
import sklearn.metrics
from lime import lime_text
from lime.lime_text import LimeTextExplainer
import sklearn
from sklearn.pipeline import make_pipeline
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sklearn.feature_extraction 
import seaborn as sns
import pickle
import datetime
from datetime import timedelta
import scipy.stats as st
from sklearn import metrics
from sklearn import svm
from sklearn.metrics import confusion_matrix, plot_confusion_matrix



Import the dataset

Choose the target features and set it to the variable 'predict'

In [ ]:
compas = pd.read_csv("compas-scores.csv")

print("Charge degrees and recidivation outcomes:", compas[["c_charge_degree", "is_recid"]].value_counts())
predict = 'is_recid'


**Error 1**

 dataset has wrong data in the form of extra categories in the features 'is_recid' and 'c_charge_degree'. These features should only have two outcomes [0,1] and ['M','F'] respectfully. however each features has one extra outcome, -1, and ). First we need to find out how many of these extra features there are and how they are related to one another in these two categories.

after confirming their existence and the amount i decided the amount was too significant to just have them deleted, and because they were not counted as recidivated it was a clerical error on the part of the office were the person was arrested but not convicted of a crime. Because of this i set the -1 value to 0 and the O value to M.

In [ ]:
os = []
total = len(compas)
for i in range(total):
  if compas[predict][i] == -1:
    os.append(compas.iloc[i])
    compas[predict][i] = 0
    print(compas[predict][i])
  if compas["c_charge_degree"][i] == "O":
    os.append(compas.iloc[i])
    compas["c_charge_degree"][i] = 'M'
    print(compas["c_charge_degree"][i])
os = pd.DataFrame(os, columns =['sex','age','race','juv_fel_count','decile_score','juv_misd_count','juv_other_count','priors_count','c_charge_degree','v_decile_score','is_recid'] )
print("number of -1: ", compas["is_recid"].value_counts())
print("recidivate decision in Os: ", o["is_recid"].value_counts())
print("O's: ", os["c_charge_degree"].value_counts())
print(compas[["c_charge_degree", "is_recid"]].value_counts())


Next, there are features that are important to the models decision making process, however they are not numbers, but rather they are string values. To make these values useable by the model we have to turn them into integers. This is a classifcation model, and there are only two values for each feature, so we will be making each faeture either 0 or a 1 for the 'sex' features and the 'c_charge_degree' feature. 

In [ ]:
#Encode sex values

print("Sex values as string:", compas['sex'].value_counts())
for f in range(total):
  if compas['sex'][f] == 'Male':
    compas['sex'][f] = 0
  if compas['sex'][f] == 'Female':
    compas['sex'][f] = 1
print("SEX VALUES AS BINARY:",compas['sex'].value_counts())
#Encode charge degree values
print("CHARGE VALUES AS STRING: ", compas['c_charge_degree'].value_counts())
for j in range(total):
  if compas['c_charge_degree'][j] == 'M':
    compas['c_charge_degree'][j] = 0
  if compas['c_charge_degree'][j] == 'F':
    compas['c_charge_degree'][j] = 1
print("CHARGE VALUES AS BINARY: ", compas['c_charge_degree'].value_counts())



**Error 2

The other error that we will fix was actually found by another author. This was that the time frame did not quite match up for both recidivate and non-recidivate defendants. For the data to be within the valid timefrane all data collection must be stopped by April 1, 2014, however there is data that continued to be collected long after, however this data is only collected from recidivate criminals and not the non-recidivate criminals. We will fix this error by dropping all of the values that appear after the cut off date.

In [ ]:
for l in range(len(compas)):
  if pd.to_datetime(compas["compas_screening_date"][l]) >= datetime.datetime(2014, 4, 1):
    compas = compas.drop([l])
print("last date of modified dataset: ", compas["compas_screening_date"].max(), "length of compas: ", len(compas))


compasfeat = compas.drop(['id', 'name', 'first', 'last','race','decile_score', 'juv_other_count','v_decile_score','compas_screening_date','dob', 'age_cat', 'days_b_screening_arrest', 
                          'c_jail_in','c_jail_out','c_case_number', 'c_offense_date', 'c_arrest_date', 'c_days_from_compas','decile_score.1','vr_case_number','vr_charge_degree', 
                          'vr_offense_date', 'vr_charge_desc','type_of_assessment','score_text', 'r_charge_desc','screening_date', 'v_type_of_assessment', 'v_score_text', 'v_screening_date', 
                          'c_charge_desc','num_r_cases','r_case_number','r_days_from_arrest','r_offense_date','r_jail_in','r_jail_out','num_vr_cases', 'is_recid','r_charge_degree','is_violent_recid' ], axis=1)


print("Null values: ", compasfeat.isnull().values.any(), "Sum: ", compasfeat.isnull().sum())

print(len(compasfeat))


**Preprocessing**

Now we are ready to preprocess the data for the model. 
First we split the data into features and the target data point.

Then we scale the data down so if there is one data point that is much larger than the others it won't skew the data too hard one way or the other. In this case that data is the age feature. all the other values are 0,1 or a value less that 10. However when it comes to age that goes even above 70, which is much higher than the other values. That is the point of scalling the data down, so that age doesn't unfairly influence that data were it shouldn't . 

Then we print the values of the scaled data as well as the mean as proof of the data's new lower scale.

We also create a paramter called "epoch"  with the value of 1000 which will be used several times in the upcoming code. This will tell us how many times the run the model in the training stage as well as help us get other necessary values later. 

In the last step of preprocessing we check for any missing values. In this case there are none, but there are several wauys to handle that scenario if it arises.



In [ ]:

x = np.array(compasfeat)
y = np.array(compas[predict])

scaler = sklearn.preprocessing.StandardScaler()
x = scaler.fit_transform(x)

epochs= 1000
print("MEAN OF SCALED DATA: ", x.mean(axis=0))
print("STANDARD DEVIATION OF SCALED DATA", x.std(axis=0))
print(len(compasfeat))


print("Null values: ", compasfeat.isnull().values.any(), "Sum: ", compasfeat.isnull().sum())


**The Model**

Now we get the meat of the work! here is where we train and test the model.

As you can see before the model "for loop" starts, there is a variable called "svmaccuracies" This is necessary later for calculatiing the confidence interval of the model. 
Underneath that variable is the variable best, where we will store the highest accuracy achieved by the model.

In side the for loop we first split the x and y into train and test categories, with four categories total.

Then we create the model by pulling an aleready made stable version of the non linear SVM model from the sklearn repository.

next we fit the data to the model and then to get an accuracy we "Score" the data and also round the data points up to two decimals which is standard. We then append all of the accuracies that we get into the svmaccuracies variable. 

underneath that is the "if-statement" for finding the highest accuracy achieved by the model. It compares the current accuracy to the highest current accuracy achieved. if the current accuracy is lower than the the highest one achieved at that point, the "best" value stays the same, however if it is higher then the best value will be replaced and the model will continue to run. 

The highest accuracy will be saved by pickling. (the code to pull the saved data back out is in side the triple astericks so the model doesn't have to be run again from the start to get the results you want later)

In [ ]:
print(compasfeat.columns)
svmaccuracies= []
best = 0
for _ in range(1000):
    x_train3, x_test3, y_train3, y_test3 = sklearn.model_selection.train_test_split(x, y, test_size=.20)
    clf= svm.NuSVC(gamma='auto', probability=True)
    clf.fit(x_train3, y_train3)
    svmacc =clf.score(x_test3, y_test3)
    svmacc = round(svmacc*100, 2)
    svmaccuracies.append(svmacc)
    if best < svmacc:
      best = svmacc
      print(best)
print("Highest Accuracy: ", best)
bigdataSVM = 'bigdataSVM.sav'
pickle.dump(clf, open(bigdataSVM, 'wb'))

#open saved model
#openbigdataSVM = pickle.load(open("bigdataSVM.sav", "rb"))



**Results**

this is where the svmaccuracies variable importance is shwon. We use that variable to get teh mean, standard diviation, as well as confidence interval of the model. the results are displated at the bottom keep the code neat in a way that i like, but it can be moved anywhere below the variables created to make the stats possible. 

next we create an i variable with a random number. this is used to get one value of the x_test and y_test datasets that will display all the prisonaer inforamtion.

we also print the confusion matrix,the LIME explainer in pyploy figure format, and the AUC for this one instance to show the results.  all of these results are also saved using 'savfig' in pdf format, however there do exists different formats as well. 

In [ ]:
#results

svmmeans = np.mean(svmaccuracies, 0)
svmstds = np.std(svmaccuracies, 0)
svmci95 = 1.96*svmstds/np.sqrt(epochs)

i = 66
print("prisoner information:\n", x_test3[i],y_test3[i])
#SVM confusion matric
svm_matrix = confusion_matrix(y_test3, clf.predict(x_test3), normalize='all')
explainer3 = lime.lime_tabular.LimeTabularExplainer(x_train3, mode='classification', feature_names = compasfeat.columns.values, class_names=['no recid', 'recid'], verbose=True, discretize_continuous=True, discretizer='quartile')

print("\nSupport vector machine accuracy: ", best, "\nprediction: ", clf.predict(x_test3[i].reshape(1, -1)), "\nPredict probability: ", clf.predict_proba(x_test3[i].reshape(1, -1)), "\nConfusion Matrix:\n", svm_matrix)

print("Support Vector Machine: \n")
svmexp = explainer3.explain_instance(x_test3[i], clf.predict_proba, num_features=len(compasfeat.columns))
#svmexp.show_in_notebook(show_table=True)
svmimg = svmexp.as_pyplot_figure()
svmimg.savefig('bigdataSVMexp', format='pdf',bbox_inches='tight')
plt.show()


y_pred_proba = clf.predict_proba(x_test3)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test3,  y_pred_proba)
auc = round(metrics.roc_auc_score(y_test3, y_pred_proba),2)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.savefig('bigdatasvmAUC.pdf', format='pdf',bbox_inches='tight')
plt.show()

print("Mean validation accuracy/loss: ", svmmeans, "stddev: ", svmstds, "Confidence Interval: ",svmci95)